In [1]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [2]:
import folium
import numpy as np
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [3]:
req_1 = requests.get(url)
soup = BeautifulSoup(req_1.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

#Convert scraped data into dataframe df

In [4]:
df = df.drop(df[df.Borough == 'Not assigned'].index)
#Drop rows with no assigned boroughs

In [5]:
df.shape
df.head(12)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Data important by csv. Code contains sensitive information and credentials, and has been hidden for security.

In [7]:
df = df.merge(coords, left_on = 'Postal code', right_on = 'Postal Code')
df = df.drop(columns='Postal Code')

In [8]:
df.head(12)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [9]:
from geopy.geocoders import Nominatim
import folium

address = 'TORONTO, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_T = location.latitude
longitude_T = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_T, longitude_T))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_T, longitude_T], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


I will be working with all of the neighborhoods, rather than only the ones with Toronto in the name.

Foursquare id code hidden for security.

In [11]:
LIMIT = 100
radius = 500

url_2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

results = requests.get(url_2).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,South St. Burger,Burger Joint,43.631314,-79.518408
1,Wingporium,Wings Joint,43.630275,-79.518169
2,Dollarama,Discount Store,43.629883,-79.518627
3,Healthy Planet,Supplement Shop,43.630214,-79.518495
4,Artisano Bakery Café,Bakery,43.631006,-79.518172


In [15]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [16]:
#Checking size of the resulting dataframe and how many venues were returned for each neighborhood.
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()

(2126, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,8,8,8,8,8,8
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,15,15,15,15,15,15


Looking at how many unique venues there are.

In [17]:
print(f"There are {len(toronto_venues['Venue Category'].unique())} unique categories.")

There are 267 unique categories.


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()
#toronto_onehot.shape

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Analysis

For this analysis, I will be replicating the same work that was performed on the New York data.

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,Alderwood / Long Branch,Pizza Place,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,Gym,Dumpling Restaurant,Eastern European Restaurant,Drugstore
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Pizza Place,Supermarket,Ice Cream Shop,Sushi Restaurant,Deli / Bodega,Diner,Restaurant,Fried Chicken Joint
3,Bayview Village,Café,Chinese Restaurant,Bank,Japanese Restaurant,Women's Store,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,Bedford Park / Lawrence Manor East,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Women's Store,Thai Restaurant,Pharmacy,Pizza Place,Pub,Café


I note that for the neighborhoods with fewer venues, when there aren't enough to fill out the top ten, the list is filled in alphabetically. It may be worth lowering the number of columns, or filling in the columns with Null. Although neighborhoods in the same situation with fewer venues may be treated similarly, I have changed the number of venues to be included to the top 5, rather than the top 10.



### Clustering the Neighborhoods
For the clustering, I have used 7 clusters. This was after adjusting the number of venues used and trying 4 and 5 clusters in the model.

In [45]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged # check the last columns!

In [46]:
#Found issue where certain neighborhoods had NaN for venues and were given a cluster label of NaN. Removing those from the dataframe.
toronto_merged.dropna(inplace=True)
#toronto_merged # Checking dataframe

In [47]:
# create map
map_clusters = folium.Map(location=[latitude_T, longitude_T], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Its interesting to see that the vast majority of the neighborhoods are in Cluster 2, and that Cluster 2 is both concentrated in the heart of Toronto and spread around it.

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,0.0,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
85,Scarborough,0.0,Park,Playground,Coffee Shop,Ethiopian Restaurant,Electronics Store,Event Space,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store
91,Downtown Toronto,0.0,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


This cluster's defining trait appears to be trails and parks. The two neighborhoods are next to the beach and near the river. I would call the cluster the Nature or Scenic cluster.

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,North York,1.0,Gym,Pizza Place,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
63,York,1.0,Grocery Store,Pizza Place,Brewery,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant
77,Etobicoke,1.0,Mobile Phone Shop,Pizza Place,Bus Line,Sandwich Place,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store


Now this is a cluster I can get behind. Pizza being either the most common or second most common venue. Pizza Cluster it is.

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2.0,Portuguese Restaurant,Coffee Shop,Intersection,Hockey Arena,French Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Dessert Shop
2,Downtown Toronto,2.0,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
3,North York,2.0,Furniture / Home Store,Clothing Store,Women's Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Boutique,Event Space,Accessories Store,Vietnamese Restaurant
4,Downtown Toronto,2.0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,College Cafeteria,Spa,Beer Bar,Sandwich Place,Burger Joint,Burrito Place
7,North York,2.0,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Asian Restaurant,Restaurant,Gym / Fitness Center,Sandwich Place,Sporting Goods Shop,Bike Shop
8,East York,2.0,Pizza Place,Gym / Fitness Center,Gastropub,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Athletics & Sports,Fast Food Restaurant
9,Downtown Toronto,2.0,Clothing Store,Coffee Shop,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Restaurant,Bubble Tea Shop,Tea Room,Italian Restaurant
10,North York,2.0,Park,Pizza Place,Pub,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
12,Scarborough,2.0,Bar,History Museum,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
13,North York,2.0,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Asian Restaurant,Restaurant,Gym / Fitness Center,Sandwich Place,Sporting Goods Shop,Bike Shop


This is the largest cluster, which contains primarily restaurants, especially Coffee Shops and Cafes.

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,3.0,Paper / Office Supplies Store,Food Service,Baseball Field,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


I'm not sure what to make of this one. Its another micro cluster, but with more commercial venues by the look of it.

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Food & Drink Shop,Park,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
21,York,4.0,Park,Pool,Women's Store,Gourmet Shop,Golf Course,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
35,East York,4.0,Park,Intersection,Convenience Store,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
49,North York,4.0,Park,Construction & Landscaping,Bakery,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
66,North York,4.0,Park,Convenience Store,Bank,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
98,Etobicoke,4.0,Park,River,Pool,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Ethiopian Restaurant,Doner Restaurant


This cluster is fairly spread out in the city, and contains mostly parks and pools. Park or Recreation Cluster would seem to be an apt name for it.

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,5.0,Fast Food Restaurant,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


The last micro cluster on the list. The most common venue being the History Museum. If I had to give a name to it, it would be the Historic Cluster.

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,Etobicoke,6.0,Baseball Field,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Field


Not sure what to say about this cluster. Baseball Fields, Women's Stores and Drugstores. Perhaps the Sports or Baseball Cluster?

### Summary
With 7 clusters, there are 3 clusters with only 1 neighborhood each. I tried both 5 and 10 columns, which shifted a few neighborhoods around, but didn't substantially change most of the clustering. I also looked at 4 or 5 clusters, but found that the micro clusters persisted, indicating that the outlying neighborhoods in these clusters are substantially different from the others. This isn't particularly relevant, but in the few times I ran the code, I found that the Pizza cluster persisted in its existence.